In [3]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded")

Libraries loaded


## Load and aggressively filter data

In [17]:
df = pd.read_csv('../data/online_retail_II.csv', encoding='ISO-8859-1')
print(f"Original: {len(df):,} rows")

df_clean = df[
    (~df['Invoice'].astype(str).str.startswith('C')) &
    (df['Quantity'] > 0) &
    (df['Price'] > 0) &
    (df['Description'].notna()) &
    (df['Country'] == 'United Kingdom')
].copy()

print(f"After cleaning + UK filter: {len(df_clean):,} rows")

df_clean['Description'] = df_clean['Description'].str.strip().str.upper()
df_clean['Description'] = df_clean['Description'].str.replace('[^A-Z0-9 ]', '', regex=True)
df_clean['Description'] = df_clean['Description'].str.replace('\s+', ' ', regex=True)

df_clean = df_clean[~df_clean['Description'].str.contains('POSTAGE|DOTCOM|BANK CHARGES|SAMPLES|ADJUST|DISCOUNT', na=False)]

print(f"Unique products: {df_clean['Description'].nunique():,}")
print(f"Unique transactions: {df_clean['Invoice'].nunique():,}")

Original: 1,067,371 rows
After cleaning + UK filter: 958,501 rows
After cleaning + UK filter: 958,501 rows
Unique products: 5,237
Unique transactions: 36,422
Unique products: 5,237
Unique transactions: 36,422


## Selecting and viewing top products

In [5]:
top_products = df_clean['Description'].value_counts().head(200)
print("Top 200 products:")
print(top_products)

print(f"\nTop product appears in: {top_products.iloc[0]:,} transactions")
print(f"#50 product appears in: {top_products.iloc[49]:,} transactions")
print(f"#100 product appears in: {top_products.iloc[99]:,} transactions")

Top 200 products:
Description
WHITE HANGING HEART TLIGHT HOLDER      5569
REGENCY CAKESTAND 3 TIER               3574
JUMBO BAG RED RETROSPOT                3163
ASSORTED COLOUR BIRD ORNAMENT          2778
PARTY BUNTING                          2598
                                       ... 
WOODEN HEART CHRISTMAS SCANDINAVIAN     829
FELTCRAFT CUSHION RABBIT                829
METAL SIGN TAKE IT OR LEAVE IT          828
TRADITIONAL WOODEN SKIPPING ROPE        827
JUMBO BAG VINTAGE LEAF                  825
Name: count, Length: 200, dtype: int64

Top product appears in: 5,569 transactions
#50 product appears in: 1,430 transactions
#100 product appears in: 1,114 transactions


In [6]:
TOP_N = 200

selected_products = df_clean['Description'].value_counts().head(TOP_N).index
df_filtered = df_clean[df_clean['Description'].isin(selected_products)].copy()

print(f"\nFiltered to {TOP_N} products:")
print(f"Transactions: {len(df_filtered):,}")
print(f"Unique invoices: {df_filtered['Invoice'].nunique():,}")
print(f"\nSelected products:")
for i, prod in enumerate(selected_products[:20], 1):
    print(f"{i:2}. {prod}")


Filtered to 200 products:
Transactions: 255,814
Unique invoices: 30,763

Selected products:
 1. WHITE HANGING HEART TLIGHT HOLDER
 2. REGENCY CAKESTAND 3 TIER
 3. JUMBO BAG RED RETROSPOT
 4. ASSORTED COLOUR BIRD ORNAMENT
 5. PARTY BUNTING
 6. LUNCH BAG BLACK SKULL
 7. LUNCH BAG SUKI DESIGN
 8. JUMBO STORAGE BAG SUKI
 9. STRAWBERRY CERAMIC TRINKET BOX
10. HEART OF WICKER SMALL
11. JUMBO SHOPPER VINTAGE RED PAISLEY
12. REX CASHCARRY JUMBO SHOPPER
13. PAPER CHAIN KIT 50S CHRISTMAS
14. HOME BUILDING BLOCK WORD
15. WOODEN FRAME ANTIQUE WHITE
16. NATURAL SLATE HEART CHALKBOARD
17. HEART OF WICKER LARGE
18. 60 TEATIME FAIRY CAKE CASES
19. WOODEN PICTURE FRAME WHITE FINISH
20. LUNCH BAG SPACEBOY DESIGN

Transactions: 255,814
Unique invoices: 30,763

Selected products:
 1. WHITE HANGING HEART TLIGHT HOLDER
 2. REGENCY CAKESTAND 3 TIER
 3. JUMBO BAG RED RETROSPOT
 4. ASSORTED COLOUR BIRD ORNAMENT
 5. PARTY BUNTING
 6. LUNCH BAG BLACK SKULL
 7. LUNCH BAG SUKI DESIGN
 8. JUMBO STORAGE BAG SUKI
 9

## Transaction basket attempt

In [7]:
basket = df_filtered.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Invoice')

basket_encoded = (basket > 0).astype(int)

print(f"Basket shape: {basket_encoded.shape}")
print(f"Sparsity: {(basket_encoded == 0).sum().sum() / (basket_encoded.shape[0] * basket_encoded.shape[1]) * 100:.1f}%")

basket_sizes = basket_encoded.sum(axis=1)
print(f"\nBasket sizes (from our {TOP_N} products):")
print(basket_sizes.describe())

basket_encoded = basket_encoded[basket_sizes >= 2]
print(f"\nAfter removing single-item baskets: {basket_encoded.shape[0]:,} transactions")

Basket shape: (30763, 200)
Sparsity: 96.1%

Basket sizes (from our 200 products):
count    30763.000000
mean         7.897182
std          9.669511
min          1.000000
25%          2.000000
50%          5.000000
75%         10.000000
max        148.000000
dtype: float64

After removing single-item baskets: 26,360 transactions


## FP-Growth run

In [15]:
#MOST TIME CONSUMING AREA. Initial runs took 20+ minutes. Currently optimized to around 8 minutes. Atleast 15 hours spent here...

In [8]:
min_support = 0.005

print(f"Running FP-Growth with min_support={min_support} ({min_support*100}%)...")
frequent_itemsets = fpgrowth(basket_encoded, min_support=min_support, use_colnames=True)

print(f"✓ Found {len(frequent_itemsets):,} frequent itemsets")

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(len)
print(f"\nItemset size distribution:")
print(frequent_itemsets['length'].value_counts().sort_index())

large_sets = frequent_itemsets[frequent_itemsets['length'] >= 3].sort_values('support', ascending=False)
print(f"\n3+ item sets: {len(large_sets)}")
if len(large_sets) > 0:
    print("\nTop 10 largest itemsets by support:")
    for idx, row in large_sets.head(10).iterrows():
        items = list(row['itemsets'])
        print(f"\nSize {len(items)}, Support: {row['support']:.4f}")
        for item in items:
            print(f"  - {item[:60]}")

Running FP-Growth with min_support=0.005 (0.5%)...
✓ Found 9,734 frequent itemsets

Itemset size distribution:
length
1     200
2    5609
3    3151
4     673
5      98
6       3
Name: count, dtype: int64

3+ item sets: 3925

Top 10 largest itemsets by support:

Size 3, Support: 0.0251
  - PINK REGENCY TEACUP AND SAUCER
  - GREEN REGENCY TEACUP AND SAUCER
  - ROSES REGENCY TEACUP AND SAUCER

Size 3, Support: 0.0204
  - JUMBO SHOPPER VINTAGE RED PAISLEY
  - JUMBO BAG RED RETROSPOT
  - JUMBO STORAGE BAG SUKI

Size 3, Support: 0.0193
  - ROSES REGENCY TEACUP AND SAUCER
  - REGENCY CAKESTAND 3 TIER
  - GREEN REGENCY TEACUP AND SAUCER

Size 3, Support: 0.0192
  - LUNCH BAG SUKI DESIGN
  - LUNCH BAG SPACEBOY DESIGN
  - LUNCH BAG BLACK SKULL

Size 3, Support: 0.0191
  - WHITE HANGING HEART TLIGHT HOLDER
  - WOODEN PICTURE FRAME WHITE FINISH
  - WOODEN FRAME ANTIQUE WHITE

Size 3, Support: 0.0187
  - LUNCH BAG SUKI DESIGN
  - LUNCH BAG CARS BLUE
  - LUNCH BAG BLACK SKULL

Size 3, Support: 0.018

## Generate association rules

In [9]:
min_confidence = 0.3

print(f"Generating rules with min_confidence={min_confidence}...")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

print(f"Generated {len(rules):,} rules")

if len(rules) > 0:
    rules['ant_len'] = rules['antecedents'].apply(len)
    rules['cons_len'] = rules['consequents'].apply(len)
    rules['total_items'] = rules['ant_len'] + rules['cons_len']
    
    print(f"\nRule statistics:")
    print(rules[['support', 'confidence', 'lift']].describe())
    
    print(f"\nRules by total items:")
    print(rules['total_items'].value_counts().sort_index())

Generating rules with min_confidence=0.3...
Generated 14,768 rules

Rule statistics:
            support    confidence          lift
count  14768.000000  14768.000000  14768.000000
mean       0.006807      0.519523     12.969205
std        0.003112      0.151680      9.342416
min        0.005008      0.300000      1.562112
25%        0.005311      0.392464      6.795523
50%        0.005842      0.494683      9.887961
75%        0.006904      0.625000     15.833775
max        0.044575      0.978571     61.268044

Rules by total items:
total_items
2     546
3    7385
4    4879
5    1816
6     142
Name: count, dtype: int64
Generated 14,768 rules

Rule statistics:
            support    confidence          lift
count  14768.000000  14768.000000  14768.000000
mean       0.006807      0.519523     12.969205
std        0.003112      0.151680      9.342416
min        0.005008      0.300000      1.562112
25%        0.005311      0.392464      6.795523
50%        0.005842      0.494683      9.88

## Try finding interesting bundles (might use this style)

In [10]:
interesting_rules = rules[
    (rules['total_items'] >= 3) &
    (rules['lift'] > 2) &
    (rules['confidence'] > 0.4)
].copy()

interesting_rules = interesting_rules.sort_values('lift', ascending=False)

print(f"Found {len(interesting_rules)} interesting bundles (3+ items, lift>2, conf>0.4)")

if len(interesting_rules) > 0:
    print("TOP 20 GIFT BUNDLES")
    
    for idx, (i, row) in enumerate(interesting_rules.head(20).iterrows(), 1):
        ant_items = list(row['antecedents'])
        cons_items = list(row['consequents'])
        
        print(f"Bundle #{idx} (Lift: {row['lift']:.2f}, Conf: {row['confidence']:.2f}, Supp: {row['support']:.4f})")
        print("If customer buys:")
        for item in ant_items:
            print(f"  ✓ {item[:70]}")
        print("They also likely buy:")
        for item in cons_items:
            print(f"  → {item[:70]}")
        print()
else:
    print("\nNo rules found with these criteria")

Found 10594 interesting bundles (3+ items, lift>2, conf>0.4)
TOP 20 GIFT BUNDLES
Bundle #1 (Lift: 61.27, Conf: 0.43, Supp: 0.0050)
If customer buys:
  ✓ PACK OF 72 RETROSPOT CAKE CASES
  ✓ WOODLAND CHARLOTTE BAG
They also likely buy:
  → CHARLOTTE BAG PINK POLKADOT
  → REGENCY CAKESTAND 3 TIER
  → CHARLOTTE BAG SUKI DESIGN

Bundle #2 (Lift: 61.27, Conf: 0.71, Supp: 0.0050)
If customer buys:
  ✓ CHARLOTTE BAG PINK POLKADOT
  ✓ REGENCY CAKESTAND 3 TIER
  ✓ CHARLOTTE BAG SUKI DESIGN
They also likely buy:
  → PACK OF 72 RETROSPOT CAKE CASES
  → WOODLAND CHARLOTTE BAG

Bundle #3 (Lift: 60.48, Conf: 0.47, Supp: 0.0050)
If customer buys:
  ✓ PACK OF 72 RETROSPOT CAKE CASES
  ✓ STRAWBERRY CHARLOTTE BAG
They also likely buy:
  → RED RETROSPOT CHARLOTTE BAG
  → REGENCY CAKESTAND 3 TIER
  → CHARLOTTE BAG SUKI DESIGN

Bundle #4 (Lift: 60.48, Conf: 0.65, Supp: 0.0050)
If customer buys:
  ✓ RED RETROSPOT CHARLOTTE BAG
  ✓ REGENCY CAKESTAND 3 TIER
  ✓ CHARLOTTE BAG SUKI DESIGN
They also likely buy:
 

## 2item pair analysis

In [11]:
pair_rules = rules[
    (rules['ant_len'] == 1) & 
    (rules['cons_len'] == 1) &
    (rules['lift'] > 1.5) &
    (rules['confidence'] > 0.3)
].copy()

pair_rules = pair_rules.sort_values('lift', ascending=False)

print(f"\nFound {len(pair_rules)} strong 2-item pairs (lift>1.5, conf>0.3)")

if len(pair_rules) > 0:
    print("TOP 2-ITEM PAIRS (High Lift Only)")
    
    for idx, (i, row) in enumerate(pair_rules.head(30).iterrows(), 1):
        item1 = list(row['antecedents'])[0]
        item2 = list(row['consequents'])[0]
        
        words1 = set(item1.split())
        words2 = set(item2.split())
        overlap = len(words1 & words2)
        
        if overlap <= 1:
            print(f"Pair #{idx}: Lift={row['lift']:.2f}, Conf={row['confidence']:.2f}")
            print(f"  {item1[:65]}")
            print(f"  + {item2[:65]}")
            print()


Found 545 strong 2-item pairs (lift>1.5, conf>0.3)
TOP 2-ITEM PAIRS (High Lift Only)


## Manual bundle categorization (potentially)

In [12]:
curated_bundles = {
    'Classic Combos': [
    ],
    'Seasonal Sets': [
    ],
    'Niche Discoveries': [
    ]
}

## Export results

In [16]:
if len(pair_rules) > 0:
    export_pairs = pair_rules.copy()
    export_pairs['antecedents'] = export_pairs['antecedents'].apply(lambda x: list(x)[0])
    export_pairs['consequents'] = export_pairs['consequents'].apply(lambda x: list(x)[0])
    
    export_pairs[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_csv(
        '../outputs/results/strong_pairs.csv', index=False
    )
    print(f"Exported {len(export_pairs)} pairs")
    print(f"Lift range: {export_pairs['lift'].min():.2f} - {export_pairs['lift'].max():.2f}")

Exported 545 pairs
Lift range: 1.56 - 18.40
